In [1]:
from ufal.udpipe import Model, Pipeline
#import os
#import re
import sys
import json
#import wget

In [2]:
#udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
#modelfile = wget.download(udpipe_url)
modelfile = 'udpipe_syntagrus.model'

In [3]:
dataFolderPath = '../data'
booksFolderPath = dataFolderPath + '/books'
booksJsonFilePath = booksFolderPath + '/books.json'
booksJsonFile = open(booksJsonFilePath);
booksJson = json.load(booksJsonFile);
booksJsonFile.close();

for bookJson in booksJson:
    print(bookJson['name'])

TODO:  12-stuljev
TODO:  pohozhdeniya-bravogo-soldata-shveyka
TODO:  zapiski sumasshedshego
TODO:  istoriya odnogo goroda
TODO:  foma-gordeev.gorod-zheltogo-djavola


In [4]:
def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken

def clean_token(token, misc):
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma

In [5]:
def process_direct(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            print('STRANGE TAG IN TAGGED!', file=sys.stderr)
            continue
        (word_id, word, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(word, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            print('NO LEMMA OR NO TOKEN!', file=sys.stderr)
            continue
        
        if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
            lemma = num_replace(token)
        spaces_after = ' '
        if 'SpacesAfter=\\s\\s\\s' in misc:
            spaces_after = '   '
        elif 'SpacesAfter=\\s\\s' in misc:
            spaces_after = '  '
        elif 'SpacesAfter=\\s\\n' in misc:
            spaces_after = '  \n'
        elif 'SpacesAfter=\\s' in misc:
            spaces_after = ' '
        elif 'SpacesAfter=\\n' in misc:
            spaces_after = '\n'        
        elif 'SpaceAfter=No' in misc:
            spaces_after = ''                    
        else:
            spaces_after = ' '
            
        chunk = word + spaces_after;
        tagged_propn.append('%s_%s~%s|' % (lemma, pos, chunk))
        
    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn


In [6]:
def lemmatize_text(text, modelfile=modelfile):    
    print('Loading the model...')
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing text...')
    output = process_direct(process_pipeline, text=text)
    print('Text done')
    return ''.join(output)

In [7]:
for bookJson in booksJson:
    name = bookJson['name']
    print('Name: ' + name)
    original_text_path = bookJson['originalTextPath']
    lemmatized_text_path = bookJson['lemmatizedTextPath']
    print('Opening original text from file... ' + original_text_path)
    original_text_file_handle = open(booksFolderPath + original_text_path, 'r', encoding='utf-8')
    original_text = original_text_file_handle.read()
    original_text_file_handle.close()
    print('Lemmatizing text...')
    lemmatized_text = lemmatize_text(original_text, modelfile)
    
    lemmatized_text_file_handle = open(booksFolderPath + lemmatized_text_path, 'w') 
    print('Writing lemmatized text to file... ' + lemmatized_text_path)
    lemmatized_text_file_handle.write(lemmatized_text)
    lemmatized_text_file_handle.close()
    print('Book done')

Name: TODO:  12-stuljev
Opening original text from file... ./txt/11. 12-stuljev.src.txt
Lemmatizing text...
Loading the model...
Processing text...
Text done
Writing lemmatized text to file... ./txt/11. 12-stuljev.lemmatized.txt
Book done
Name: TODO:  pohozhdeniya-bravogo-soldata-shveyka
Opening original text from file... ./txt/12. pohozhdeniya-bravogo-soldata-shveyka.src.txt
Lemmatizing text...
Loading the model...
Processing text...


NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA OR NO TOKEN!
NO LEMMA O

Text done
Writing lemmatized text to file... ./txt/12. pohozhdeniya-bravogo-soldata-shveyka.lemmatized.txt
Book done
Name: TODO:  zapiski sumasshedshego
Opening original text from file... ./txt/13. zapiski sumasshedshego.src.txt
Lemmatizing text...
Loading the model...
Processing text...
Text done
Writing lemmatized text to file... ./txt/13. zapiski sumasshedshego.lemmatized.txt
Book done
Name: TODO:  istoriya odnogo goroda
Opening original text from file... ./txt/14. istoriya odnogo goroda.src.txt
Lemmatizing text...
Loading the model...
Processing text...
Text done
Writing lemmatized text to file... ./txt/14. istoriya odnogo goroda.lemmatized.txt
Book done
Name: TODO:  foma-gordeev.gorod-zheltogo-djavola
Opening original text from file... ./txt/5. foma-gordeev.gorod-zheltogo-djavola.src.txt
Lemmatizing text...
Loading the model...
Processing text...
Text done
Writing lemmatized text to file... ./txt/5. foma-gordeev.gorod-zheltogo-djavola.lemmatized.txt
Book done
